In [120]:
import os
import pandas as pd
import json
import folium
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta
%matplotlib inline  

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO



In [142]:
transactions = pd.read_csv('../../data/transactions.small.csv')
transactions = transactions.rename(columns={'id':'tid'})
transactions['month'] = transactions['date'].apply(lambda x: int(x.split('/')[0]))
transactions['day'] = transactions['date'].apply(lambda x: int(x.split('/')[1]))
transactions['year'] = transactions['date'].apply(lambda x: int(x.split('/')[2]))
transactions['time'] = pd.to_datetime(transactions['time'])
transactions['amount'] = transactions.amount.astype(float)

In [143]:
transactions.head()

,tid,source,target,date,time,amount,currency,month,day,year
0,7d5d4762-24fe-4cf0-8185-40eff4d299a0,ee288787-e310-4bb8-b2a1-f0f6a1ad8f06,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,2017-11-11 15:57:52,1683.730271,None,24,9,2015
1,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,27/17/2015,2017-11-11 23:59:57,733.476914,None,27,17,2015
2,9adcfd16-ac51-47e7-85e8-8dab05ddbc23,cc9ec01d-9414-413e-bf97-896271fdc446,4c3809e8-6d31-4e79-8abd-f933dcd95c08,27/13/2017,2017-11-11 15:34:34,469.263757,None,27,13,2017
3,529d9a6a-8512-4ff9-b7e9-dcd1674fbfc7,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,ac0c5a47-503a-4852-ac0b-b233239ede95,24/09/2015,2017-11-11 15:57:52,1683.730271,None,24,9,2015
4,56ce6a32-4059-4e35-8650-60364c0199f4,ada7124c-fea2-4956-8699-759b7b6505f3,a397e9d5-259a-4931-a073-50b007ccf1df,27/03/2015,2017-11-11 23:23:42,3737.245020,None,27,3,2015


In [131]:
clients = pd.read_csv('../../data/clients.small.csv')
clients.head()

,id,first_name,last_name,age,email,occupation,political_views,nationality,university,academic_degree,address,postal_code,country,city
0,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,865 Brotherhood Mall,85875,Luxembourg,Bradenton
1,860c16c2-a66f-4dca-8060-89396a5c41c1,Madlyn,Nunez,43,bazooka1835@live.com,Wood Worker,Socialist,Jordanian,University of California Santa Barbara (UC Sa...,None,1310 Northridge Path,85875,Moldova,Gulfport
2,fec91c68-5577-4979-b522-6804a0efd371,Towanda,Griffin,38,algerine1969@gmail.com,Ironmonger,Liberal,Iranian,University of California Riverside (UC Rivers...,None,922 Terrace Grove,85875,Antarctica,Carpentersville
3,09f7f81c-e8c3-4fb7-98cb-70f04d1df9b4,Dyan,Mcconnell,36,abroad1911@yahoo.com,Aircraft Maintenance Engineer,Socialist,Uruguayan,Georgia Institute of Technology (Georgia Tech),None,1274 Rodgers Townline,85875,Belgium,Goodlettsville
4,010b4fa3-cf0b-4cf5-9f95-aab959ac4c8c,Cristin,Ross,51,qualm1898@live.com,Chauffeur,Libertarian,Polish,University of California San Diego (UC San Di...,None,1219 Ringold Pike,85875,St. Lucia,Albany


In [132]:
companies = pd.read_csv('../../data/companies.small.csv')
companies.head()

,id,type,name,country
0,9c6e1a65-9239-44d6-80cc-c3b027a92372,Professional corporation,Body Fate,Japan
1,32ba86c3-dc64-4e6d-b79b-372a0d52c26e,Incorporated,Blyth Inc.,Central African Republic
2,e1a6ffb8-46fa-488a-ac66-2d5db12a7f0f,Limited Liability Partnership,Naturohair,American Samoa
3,a2a70edc-7895-461c-94c8-4d8d0cedff47,Limited Liability Company,Strongbod,South Sudan
4,ffc343f8-0938-43c4-aa9e-98cd052f9355,Limited Liability Partnership,Matrix Architectural Service,Malaysia


In [133]:
atms = pd.read_csv('../../data/atms.small.csv')
atms.head()

,id,latitude,longitude
0,7c22e035-e905-4be2-affa-130489b36203,21.108465,36.738927


In [134]:
atm_transactions = transactions.merge(atms, left_on = 'target', right_on='id')

In [43]:
transactions.merge(atms, left_on = 'source', right_on='id')

,tid,source,target,date,time,amount,currency,month,day,id,latitude,longitude


In [44]:
incoming = clients.merge(transactions, left_on='id', right_on='target')

In [179]:
incoming.head()

,id,first_name,last_name,age,email,occupation,political_views,nationality,university,academic_degree,...,city,tid,source,target,date,time,amount,currency,month,day
0,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,d21943b3-7404-47ad-9a07-497791c285ff,969b0731-ea5b-4b0e-9685-2e4cdf560e2c,b07102b8-5373-4d03-8eae-d68b3cc407e0,25/27/2015,19:57:42,4926.467008,None,27,25
1,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,5ff72dac-bcdf-4cec-91de-1efde4bc3a21,c7961e42-0a32-4a0a-921b-762c1c8998f7,b07102b8-5373-4d03-8eae-d68b3cc407e0,01/23/2016,20:50:19,5.061788,None,23,1
2,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,d030b4b1-49b0-4376-ae1f-b5c60b638d69,bc6b228b-5021-4b2d-859b-8c0ab3cbf11e,b07102b8-5373-4d03-8eae-d68b3cc407e0,05/18/2015,06:31:06,7.533453,None,18,5
3,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,b8186285-2ee6-4609-9709-fb88437856ba,a3d9cdd5-bdec-4b66-ab91-a751556a1527,b07102b8-5373-4d03-8eae-d68b3cc407e0,01/19/2017,13:56:57,9.459083,None,19,1
4,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,8d0b4634-aa4a-45d0-9a26-eab502b0e9f2,a3d9cdd5-bdec-4b66-ab91-a751556a1527,b07102b8-5373-4d03-8eae-d68b3cc407e0,05/18/2017,04:15:27,20.609939,None,18,5


In [46]:
outgoing = clients.merge(transactions, left_on='id', right_on='source')

In [178]:
outgoing.head()

,id,first_name,last_name,age,email,occupation,political_views,nationality,university,academic_degree,...,city,tid,source,target,date,time,amount,currency,month,day
0,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,aa28ce5f-e0d4-4ac8-9406-f882dd5771f2,b07102b8-5373-4d03-8eae-d68b3cc407e0,7bbcab8f-6972-4580-97b4-d23cefe83fc4,25/27/2015,19:57:42,4485.517388,None,27,25
1,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,08cb5e6c-0e5c-4640-b578-5aaa229d129b,b07102b8-5373-4d03-8eae-d68b3cc407e0,2a59959b-1ac0-4a9a-a8a8-35582e36ae98,05/25/2016,13:17:37,6.183723,None,25,5
2,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,50bb7ebc-19b0-4b5e-9e27-4cf91680d7a8,b07102b8-5373-4d03-8eae-d68b3cc407e0,2a59959b-1ac0-4a9a-a8a8-35582e36ae98,03/27/2015,05:36:15,2.243495,BZD,27,3
3,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,2cc61bee-13b1-4758-ad36-e4ce5cb147a0,b07102b8-5373-4d03-8eae-d68b3cc407e0,2a59959b-1ac0-4a9a-a8a8-35582e36ae98,10/23/2017,05:34:17,5.942126,BBD,23,10
4,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,d73ca4a7-86ad-4955-8714-4fa528d6a72f,b07102b8-5373-4d03-8eae-d68b3cc407e0,2a59959b-1ac0-4a9a-a8a8-35582e36ae98,10/02/2017,02:48:21,63.746358,None,2,10


In [177]:
pd.concat((outgoing,incoming), axis=0).head()

,id,first_name,last_name,age,email,occupation,political_views,nationality,university,academic_degree,...,city,tid,source,target,date,time,amount,currency,month,day
0,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,aa28ce5f-e0d4-4ac8-9406-f882dd5771f2,b07102b8-5373-4d03-8eae-d68b3cc407e0,7bbcab8f-6972-4580-97b4-d23cefe83fc4,25/27/2015,19:57:42,4485.517388,None,27,25
1,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,08cb5e6c-0e5c-4640-b578-5aaa229d129b,b07102b8-5373-4d03-8eae-d68b3cc407e0,2a59959b-1ac0-4a9a-a8a8-35582e36ae98,05/25/2016,13:17:37,6.183723,None,25,5
2,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,50bb7ebc-19b0-4b5e-9e27-4cf91680d7a8,b07102b8-5373-4d03-8eae-d68b3cc407e0,2a59959b-1ac0-4a9a-a8a8-35582e36ae98,03/27/2015,05:36:15,2.243495,BZD,27,3
3,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,2cc61bee-13b1-4758-ad36-e4ce5cb147a0,b07102b8-5373-4d03-8eae-d68b3cc407e0,2a59959b-1ac0-4a9a-a8a8-35582e36ae98,10/23/2017,05:34:17,5.942126,BBD,23,10
4,b07102b8-5373-4d03-8eae-d68b3cc407e0,Kermit,Reynolds,63,epical1905@yandex.com,Warden,Socialist,Estonian,University of Massachusetts Boston (UMass Boston),None,...,Bradenton,d73ca4a7-86ad-4955-8714-4fa528d6a72f,b07102b8-5373-4d03-8eae-d68b3cc407e0,2a59959b-1ac0-4a9a-a8a8-35582e36ae98,10/02/2017,02:48:21,63.746358,None,2,10


Here, we join the transactions target to source: each row in the result will have a pair of transactions with the target of the first one equal to the source of the second. Then we calculate the time difference between the two and filter out those with a great time or amount difference. Then we can get all of these pairs, or only those that have a later (or earlier) in time (time of incoming transaction) than their out time (time of outgoing transaction), or similarly for the amounts.

Next up is actually counting those that have some spread: one in, several out, such that the sum of the outs is close to the in. Or vice versa.

In [180]:
in_and_out = transactions.merge(transactions, left_on='target', right_on='source')
in_and_out.head()

,tid_x,source_x,target_x,date_x,time_x,amount_x,currency_x,month_x,day_x,year_x,tid_y,source_y,target_y,date_y,time_y,amount_y,currency_y,month_y,day_y,year_y
0,7d5d4762-24fe-4cf0-8185-40eff4d299a0,ee288787-e310-4bb8-b2a1-f0f6a1ad8f06,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,2017-11-11 15:57:52,1683.730271,None,24,9,2015,3bd8356e-84e5-48f6-87d3-de6346d6eeab,4abed95d-5618-42e8-a5a0-06e4d69f81ca,e9528bb7-c06f-4df4-acf9-a50e85d01e9c,24/09/2015,2017-11-11 15:57:52,6738.948250,None,24,9,2015
1,7d5d4762-24fe-4cf0-8185-40eff4d299a0,ee288787-e310-4bb8-b2a1-f0f6a1ad8f06,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,2017-11-11 15:57:52,1683.730271,None,24,9,2015,467cfc2d-9478-4def-8a6e-59bc41f98af3,4abed95d-5618-42e8-a5a0-06e4d69f81ca,63196395-e25f-4f07-b35e-5dd0f9abc034,24/09/2015,2017-11-11 15:57:52,6738.948250,None,24,9,2015
2,7d5d4762-24fe-4cf0-8185-40eff4d299a0,ee288787-e310-4bb8-b2a1-f0f6a1ad8f06,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,2017-11-11 15:57:52,1683.730271,None,24,9,2015,9517a8dc-5271-4567-8f51-e02848e3dda3,4abed95d-5618-42e8-a5a0-06e4d69f81ca,0c1f1a12-6637-4979-8d70-34c044ea2131,08/28/2016,2017-11-11 07:41:36,46.805851,None,8,28,2016
3,7d5d4762-24fe-4cf0-8185-40eff4d299a0,ee288787-e310-4bb8-b2a1-f0f6a1ad8f06,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,2017-11-11 15:57:52,1683.730271,None,24,9,2015,0bb70b8f-a363-4737-81e2-dea53c00cca3,4abed95d-5618-42e8-a5a0-06e4d69f81ca,0c1f1a12-6637-4979-8d70-34c044ea2131,10/14/2017,2017-11-11 16:07:22,0.171252,None,10,14,2017
4,7d5d4762-24fe-4cf0-8185-40eff4d299a0,ee288787-e310-4bb8-b2a1-f0f6a1ad8f06,4abed95d-5618-42e8-a5a0-06e4d69f81ca,24/09/2015,2017-11-11 15:57:52,1683.730271,None,24,9,2015,4f35236c-5c52-4faf-8f0b-be94db3772c4,4abed95d-5618-42e8-a5a0-06e4d69f81ca,0c1f1a12-6637-4979-8d70-34c044ea2131,08/05/2016,2017-11-11 22:12:10,3.453400,None,8,5,2016


In [155]:
in_and_out_f1 = in_and_out.copy()
in_and_out_f1['timediff'] = in_and_out['time_x'] - in_and_out['time_y']
in_and_out_f1['timediff'] = in_and_out_f1['timediff'].apply(lambda x: timedelta.total_seconds(x))
in_and_out_f1['timediff'] = in_and_out_f1['timediff'] + 24*3600*(in_and_out_f1['day_x']-in_and_out_f1['day_y']) + 30*24*3600*(in_and_out_f1['month_x']-in_and_out_f1['month_y']) + 365*30*24*3600*(in_and_out_f1['year_x']-in_and_out_f1['year_y'])
in_and_out_f1['abstimediff'] = abs(in_and_out_f1['timediff'])

In [191]:
#Nonnegative timediff means this client first sent stuff and then received stuff.
in_and_out_f2 = in_and_out_f1[(in_and_out_f1.abstimediff < 24*3600) & (in_and_out_f1.timediff >= 0)]
in_and_out_f2 = in_and_out_f2[(abs(in_and_out_f2.amount_x  - in_and_out_f2.amount_y) < 0.1*in_and_out_f2.amount_x) & (in_and_out_f2.amount_x <= in_and_out_f2.amount_y)]

In [194]:
shady_deals = in_and_out_f2[['tid_x', 'tid_y', 'source_x', 'target_x', 'target_y','timediff','amount_x','amount_y']]
print(shady_deals.shape)
shady_deals.drop_duplicates(subset=['tid_x'])

(264, 8)


,tid_x,tid_y,source_x,target_x,target_y,timediff,amount_x,amount_y
295,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,ac24024f-32b6-4f51-b608-2ef34d4fbe7d,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,1c0a994a-f99e-4190-aac2-72811289b510,0.0,733.476914,733.758586
419,d6d1a0a5-3922-4ac1-8488-57272ee75840,ac24024f-32b6-4f51-b608-2ef34d4fbe7d,38b9adea-6e64-4124-b02d-fda88e72dabb,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,1c0a994a-f99e-4190-aac2-72811289b510,0.0,733.476914,733.758586
481,7b769481-8855-427d-a2c2-dd46882c0c16,ac24024f-32b6-4f51-b608-2ef34d4fbe7d,8fc0094c-79f9-48e7-9ef2-2489d0e795d4,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,1c0a994a-f99e-4190-aac2-72811289b510,0.0,733.476914,733.758586
543,c8ee6bea-4f91-40b0-a569-40a5611e6edd,ac24024f-32b6-4f51-b608-2ef34d4fbe7d,74376ba2-7b19-4a3b-a165-d8f34c1d0f9b,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,1c0a994a-f99e-4190-aac2-72811289b510,0.0,733.476914,733.758586
574,8fad6b1c-3aa3-45ce-8368-28c34f28404e,ac24024f-32b6-4f51-b608-2ef34d4fbe7d,4c3809e8-6d31-4e79-8abd-f933dcd95c08,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,1c0a994a-f99e-4190-aac2-72811289b510,0.0,733.476914,733.758586
9131,96afc3a5-107a-451f-80fa-3c59c92948e4,2bf1e9f2-830d-4f8e-a235-141ac356d764,4c3809e8-6d31-4e79-8abd-f933dcd95c08,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,24300436-a977-4f49-a732-a3661c92a4f1,0.0,733.476914,733.758586
9209,54a472ea-1e99-41b8-b87e-c339bcd2a116,2bf1e9f2-830d-4f8e-a235-141ac356d764,8fc0094c-79f9-48e7-9ef2-2489d0e795d4,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,24300436-a977-4f49-a732-a3661c92a4f1,0.0,733.476914,733.758586
9365,aac3cce0-0314-42c0-bca7-da512101b091,2bf1e9f2-830d-4f8e-a235-141ac356d764,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,24300436-a977-4f49-a732-a3661c92a4f1,0.0,733.476914,733.758586
9495,760083ef-d9e6-4d70-9efc-4953cff45149,2bf1e9f2-830d-4f8e-a235-141ac356d764,74376ba2-7b19-4a3b-a165-d8f34c1d0f9b,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,24300436-a977-4f49-a732-a3661c92a4f1,0.0,733.476914,733.758586
9547,7176ceff-52a9-4cd6-9c75-96f402d09f13,2bf1e9f2-830d-4f8e-a235-141ac356d764,38b9adea-6e64-4124-b02d-fda88e72dabb,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,24300436-a977-4f49-a732-a3661c92a4f1,0.0,733.476914,733.758586


In [193]:
shady_deals.drop_duplicates(subset=['tid_y'])

,tid_x,tid_y,source_x,target_x,target_y,timediff,amount_x,amount_y
295,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,ac24024f-32b6-4f51-b608-2ef34d4fbe7d,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,1c0a994a-f99e-4190-aac2-72811289b510,0.0,733.476914,733.758586
297,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,a904480f-c9f7-4186-b33a-dad8b0bc14e9,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,d341bdc3-90dd-445b-a6c2-71a0a51feb97,0.0,733.476914,733.758586
301,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,e3f4c462-8e12-4795-a49e-edac60736c7e,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,24300436-a977-4f49-a732-a3661c92a4f1,0.0,733.476914,733.758586
302,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,e7bb4abe-20bb-499d-b21d-0158e1aa5d5a,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,e2dbc017-597e-429e-8efc-2a4c93144220,0.0,733.476914,733.758586
304,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,5b9b292c-e60b-4a3e-b521-9132289b355f,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,b80a5421-d4be-471f-8336-9ba3674b59c8,0.0,733.476914,733.758586
308,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,17decde2-90f7-499f-b96a-76aff5f4d681,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,776cefb5-b6af-451a-88d3-e195291bffa2,0.0,733.476914,733.758586
309,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,a6185628-b219-421d-a724-6f6a2a0970d2,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,b4315d5d-4650-441b-949f-1ee2beafed32,0.0,733.476914,733.758586
310,357a8314-c6b2-4a38-b6df-3bcaa4b6ecbb,24f7b54f-cd16-4a08-ad56-5fe979865a1e,e5018dfa-d6fc-4f44-a4ae-f59cd9e3275a,e0708e37-8466-4893-bc1b-bbae9fdd9aeb,0f1c31f9-1be1-4f4f-940e-95e1471a5fe3,0.0,733.476914,733.758586
9131,96afc3a5-107a-451f-80fa-3c59c92948e4,2bf1e9f2-830d-4f8e-a235-141ac356d764,4c3809e8-6d31-4e79-8abd-f933dcd95c08,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,24300436-a977-4f49-a732-a3661c92a4f1,0.0,733.476914,733.758586
9135,96afc3a5-107a-451f-80fa-3c59c92948e4,3a754d5e-6a50-4a6a-9c65-734be481afc7,4c3809e8-6d31-4e79-8abd-f933dcd95c08,e4ad5cec-82c0-4e5e-85a2-f2a2e7bfcf52,0f1c31f9-1be1-4f4f-940e-95e1471a5fe3,0.0,733.476914,733.758586
